Using this to plot distributions of reaction centres of TS structures from initial model (D_init), final model (D,after NLS optimisation), ground truth, and linear approximation.

In [1]:
from rdkit import Chem, Geometry
from rdkit.Chem import AllChem

import numpy as np
from scipy import stats
from scipy.optimize import least_squares
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# sys.path.insert(0, "Users/rmhavij/ts_gen/create_figs/scripts/")
# from gaussian import Gaussian

In [3]:
# base_folder = r'../data/raw/' # normal
base_folder = 'Users/rmhavij/3d-reactions/data/raw/' # azure

test_ts_file = base_folder + 'test_ts.sdf'
reactant_file = base_folder + 'test_reactants.sdf'
product_file = base_folder + 'test_products.sdf'
model_ts_no_opt_file = base_folder + 'model_ts.sdf'

test_r = Chem.SDMolSupplier(reactant_file, removeHs=False, sanitize=False)
test_r = [x for x in test_r]

test_ts = Chem.SDMolSupplier(test_ts_file, removeHs=False, sanitize=False)
test_ts = [ts for ts in test_ts]

test_p = Chem.SDMolSupplier(product_file, removeHs=False, sanitize=False)
test_p = [x for x  in test_p]

# will need to add my non-optimised versions later
# model_ts_no_opt = Chem.SDMolSupplier(model_ts_no_opt_file, removeHs=False, sanitize=False)
# model_ts_no_opt = [x for x  in model_ts_no_opt]

In [7]:
# temp cos it's easier for now
D_init = np.load('Users/rmhavij/ts_gen/create_figs/model_data/D_init.npy')
av_mit_d_init = np.load('Users/rmhavij/ts_gen/azure-experiments/log/21May28_0634AM/D_init.npy')

In [ ]:
# can optimise the D_init here OR could save D_init and optimised version in function

In [ ]:
# all for reaction core

ground_truth = []  
linear_approx = []
mit_d_init = []
mit_d_final = []
my_d_init = [] # feed my created D_init into this (e.g. from modifying their model, my own model, etc.)

for idx in range(len(test_ts)):
    
    # get N and create mask
    N = test_ts[idx].GetNumAtoms()
    core_mask = (Chem.GetAdjacencyMatrix(test_p[idx]) + Chem.GetAdjacencyMatrix(test_r[idx])) == 1
    
    # need to modify model_ts_no_opt 
    ground_truth.append(np.ravel(Chem.Get3DDistanceMatrix(test_ts[idx]) * core_mask))
    linear_approx.append(np.ravel( (Chem.Get3DDistanceMatrix(test_r[idx]) + Chem.Get3DDistanceMatrix(test_p[idx])) / 2 * core_mask))
    mit_d_init.append(np.ravel(D_init[idx][0:N,0:N] * core_mask))
    mit_d_final.append(np.ravel(Chem.Get3DDistanceMatrix(model_ts_no_opt[idx]) * core_mask))
    my_d_init.append(np.ravel(av_mit_d_init[idx][0:N,0:N] * core_mask)) 

# concat + ravel
ground_truth_all = np.concatenate(ground_truth).ravel()
linear_approx_all = np.concatenate(linear_approx).ravel()
mit_d_init_all = np.concatenate(mit_d_init).ravel()
mit_d_final_all = np.concatenate(mit_d_final).ravel() 
my_d_init_all = np.concatenate(my_d_init).ravel()

# no nulls TODO: put this in a loop to align if any null
ground_truth_all = ground_truth_all[ground_truth_all != 0]
linear_approx_all = linear_approx_all[linear_approx_all != 0]
mit_d_init_all = mit_d_init_all[mit_d_init_all != 0]
mit_d_final_all = mit_d_final_all[mit_d_final_all != 0] 
my_d_init_all = my_d_init_all[my_d_init_all != 0]